In [8]:
import pandas as pd
import numpy as np
import duckdb

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score,make_scorer, f1_score
from sklearn.model_selection import GridSearchCV, StratifiedKFold,cross_val_predict
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder

import xgboost as xgb
import lightgbm as lgb

import mlflow
import mlflow.sklearn

import joblib
import os
import shutil


# Carregamento dos dados

In [ ]:
# Caminho do banco
db_path = "../../data/duckdb/database.duckdb"

# Conexão com o banco DuckDB
con = duckdb.connect(db_path)

# Carrega os dados da camada bronze
clientes_df = con.execute("SELECT * FROM silver.clientes").df()
consumo_df = con.execute("SELECT * FROM silver.consumo").df()

df = consumo_df.merge(clientes_df, on='client_id', how='inner')

,client_id,date,consumption_kwh,region
0,C0000,2023-01-01,18.64,Norte
1,C0000,2023-01-02,16.63,Norte
2,C0000,2023-01-03,18.11,Norte
3,C0000,2023-01-04,18.25,Norte
4,C0000,2023-01-05,19.81,Norte
5,C0000,2023-01-06,15.87,Norte
6,C0000,2023-01-07,20.30,Norte
7,C0000,2023-01-08,19.35,Norte
8,C0000,2023-01-09,18.30,Norte
9,C0000,2023-01-10,13.34,Norte


# Feature Engineering

In [10]:
# Feature engineering por client_id
class TemporalFeatureExtractor(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    # é necessário implementar os métodos fit para que o objeto possa ser usado em um pipeline
    def fit(self, X, y=None):
        return self

    # o método transform define como os dados serão transformados
    def transform(self, X):

        if not isinstance(X, pd.DataFrame):
            raise ValueError("Input must be a pandas DataFrame")
        
        df = X.copy()

        # Garantir que a coluna 'date' esteja no formato datetime
        df["date"] = pd.to_datetime(df["date"])
        
        def _compute(group):
            # Garantir a série temporal ordenada
            ts = group.sort_values("date")
            

            vals = ts["consumption_kwh"].values
            times = ts["date"].astype(np.int64).values.reshape(-1, 1)
            
            # Calcular estatísticas básicas
            stats = {
                "mean": vals.mean(),
                "median": np.median(vals),
                "std": vals.std(ddof=0),
                "max": vals.max(),
                "min": vals.min(),
            }
            
            # A regressão linear modelará a relação entre tempo e consumo de energia. O objetivo é entender como o consumo está evoluindo ao longo do tempo.
            lr = LinearRegression().fit(times, vals)

            # O slope (coeficiente angular da reta) é a taxa de variação do consumo em relação ao tempo:
            # slope > 0 → tendência de crescimento
            # slope < 0 → tendência de queda
            # slope = 0 → consumo constante ao longo do tempo
            
            stats["slope"] = lr.coef_[0]
            
            return pd.Series(stats)
        
        features = df.groupby("client_id").apply(_compute).reset_index()
        return features

In [11]:
# Extrair features
fe = TemporalFeatureExtractor()
feat_df = fe.transform(df)

feat_df = feat_df.merge(clientes_df[["client_id", "region"]], on="client_id", how="inner")

C:\Users\jataf\AppData\Local\Temp\ipykernel_35976\3221286679.py:50: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  features = df.groupby("client_id").apply(_compute).reset_index()


# Divisão de dados para inferência

In [12]:
previsao_df = feat_df[feat_df['region'] == 'Desconhecida']

X_pred = previsao_df.drop(columns=['client_id', 'region'])

# Inferência

In [17]:
local_path = "../../models/region_classificacao/local_model"

# Carregando modelo
model = mlflow.sklearn.load_model(local_path)
# Carregando o label encoder
le_carregado = joblib.load("../../models/region_classificacao/label_encoder_classificador/label_encoder.pkl")

In [18]:
# Predição
y_pred = model.predict(X_pred)

y_pred = le_carregado.inverse_transform(y_pred)

# Resultado final
output_df = pd.DataFrame({
    "client_id": previsao_df["client_id"],
    "region": y_pred
})

print(output_df.head())

   client_id  region
15     C0015  Centro
50     C0050   Norte
58     C0058     Sul
77     C0077   Norte
92     C0092   Norte


# Salvar Resultados

In [19]:
# Cria tabelas dentro do schema output
con.execute("""
CREATE TABLE IF NOT EXISTS output.region_classificador (
    client_id VARCHAR,
    region VARCHAR
)
""")

In [20]:
# Limpa dados se as tabelas já existirem
con.execute("DELETE FROM output.region_classificador")

In [21]:
# Registra como tabelas temporárias
con.register("output_df", output_df)

In [22]:
# Insere os dados nas tabelas output
con.execute("INSERT INTO output.region_classificador SELECT * FROM output.region_classificador")